In [1]:
import config
from tqdm import tqdm
import numpy as np
import torch
import modules.models as models
import modules.dataloaders as detection_data_loader
import modules.metrics as metrics

from torchinfo import summary

INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.14 (you have 1.4.10). Upgrade using: pip install --upgrade albumentations


# Model Setup

## BED manual optim trained with FASDD

In [2]:
bed_model = models.SIMPLE_BED_DETECTOR().to(config.DEVICE)

## SVD

In [3]:
svd_model = models.SVD_BED_DETECTOR().to(config.DEVICE)

## Pruned after SVD

In [4]:
pruned_after_svd_model = models.PRUNED_AFTER_SVD_BED_DETECTOR().to(config.DEVICE)

### Load Checkpoint

In [5]:
def load_checkpoint(model_path, model, optimizer, scheduler, device):
    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    if optimizer is not None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    if scheduler is not None:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    start_epoch = checkpoint['epoch']
    print(f"Loading Model. Trained during {start_epoch} epochs")
    return start_epoch

## BED FASDD

In [6]:
bed_folder = './experiments/test_20_no_sigmoid_softmax_permute_out/weights/'
bed_weights = bed_folder + 'BED_detector__best_mAP=0.6405__epoch=144.pt'

In [7]:
bed_model.eval()
load_checkpoint(
    bed_weights,
    bed_model,
    None,
    None,
    'cuda')
bed_model.to('cpu')
print("Model forced to eval")

Loading Model. Trained during 144 epochs
Model forced to eval


## BED SVD

In [8]:
svd_folder = './experiments/test_31_svd_080_simple_model/weights/'
svd_weights = svd_folder + 'BED_detector__best_mAP=0.6204__epoch=4.pt'

In [9]:
svd_model.eval()
load_checkpoint(
    svd_weights,
    svd_model,
    None,
    None,
    'cuda')
svd_model.to('cpu')
print("Model forced to eval")

Loading Model. Trained during 4 epochs
Model forced to eval


## Pruned after SVD

In [10]:
pruned_after_svd_folder = './experiments/test_35_pruning_090_after_svd_080_simple_model_more_train/weights/'
pruned_after_svd_weights = pruned_after_svd_folder + 'BED_detector__best_mAP=0.6289__epoch=14.pt'

In [11]:
pruned_after_svd_model.eval()
load_checkpoint(
    pruned_after_svd_weights,
    pruned_after_svd_model,
    None,
    None,
    'cuda')
pruned_after_svd_model.to('cpu')
print("Model forced to eval")

Loading Model. Trained during 14 epochs
Model forced to eval


### Torch Summary

In [12]:
print(summary(bed_model, input_size=(1, 3, 224, 224)))
print(summary(svd_model, input_size=(1, 3, 224, 224)))
print(summary(pruned_after_svd_model, input_size=(1, 3, 224, 224)))

Layer (type:depth-idx)                   Output Shape              Param #
SIMPLE_BED_DETECTOR                      [1, 12, 7, 7]             --
├─Sequential: 1-1                        [1, 12, 7, 7]             --
│    └─Conv2d: 2-1                       [1, 32, 224, 224]         864
│    └─BatchNorm2d: 2-2                  [1, 32, 224, 224]         64
│    └─ReLU: 2-3                         [1, 32, 224, 224]         --
│    └─Dropout2d: 2-4                    [1, 32, 224, 224]         --
│    └─MaxPool2d: 2-5                    [1, 32, 112, 112]         --
│    └─Conv2d: 2-6                       [1, 16, 112, 112]         4,608
│    └─BatchNorm2d: 2-7                  [1, 16, 112, 112]         32
│    └─ReLU: 2-8                         [1, 16, 112, 112]         --
│    └─Dropout2d: 2-9                    [1, 16, 112, 112]         --
│    └─MaxPool2d: 2-10                   [1, 16, 56, 56]           --
│    └─Conv2d: 2-11                      [1, 16, 56, 56]           256
│    └─Bat

# Eval Epoch before Export

In [13]:
def eval_fn(loader, model, metric, device):
    
    model.to(device)
    model.eval()
    loop = tqdm(loader, desc='Validating', leave=True)

    for batch_idx, (x, y) in enumerate(loop):
        x, y = x.to(device), y.to(device)
        out = model(x)
        
        # Remove Permute from the model
        out = out.permute(0, 2, 3, 1)
                
        # Mean Average Precision
        for idx in range(x.shape[0]):
            target_boxes = metrics.get_true_boxes(y[idx].detach().to('cpu'))
            pred_boxes = metrics.get_pred_boxes(out[idx].detach().to('cpu'))
            metric.update(preds = pred_boxes, target = target_boxes) 

    meanAP = metric.compute()
    metric.reset()
    print(f'Val mAP = {meanAP["map_50"]:.4f}') 

    return (
        {'mAP': meanAP['map_50'],
         'AP': [meanAP['map_per_class'][0].item(), meanAP['map_per_class'][1].item()],
         'AR': [meanAP['mar_100_per_class'][0].item(), meanAP['mar_100_per_class'][1].item()]
        }
    )

In [14]:
detection_loader = detection_data_loader.get_val_loader()


TEST DFire dataset
DFire Removed wrong images: 0
DFire Removed due to overlapping: 310
DFire Removed due to more than 10: 13

Test DFire dataset len: 3983

TEST FASDD UAV dataset
FASDD Removed wrong images: 0
FASDD Removed due to overlapping: 377
FASDD Removed due to more than 10: 156

Test FASDD UAV dataset len: 3648

TEST FASDD CV dataset
FASDD Removed wrong images: 0
FASDD Removed due to overlapping: 317
FASDD Removed due to more than 10: 44

Test FASDD CV dataset len: 15523

Concatenate Test DFire and FASDD UAV datasets
Test dataset len: 7631
Concatenate with FASDD CV dataset
Test dataset len: 23154


In [15]:
with torch.no_grad():
    val_metrics = eval_fn(
        loader=detection_loader, 
        model=bed_model,                         
        metric=metrics.map_metric,
        device=config.DEVICE)
    print(f'BED FASDD\n{val_metrics}')
    val_metrics = eval_fn(
        loader=detection_loader, 
        model=svd_model,                         
        metric=metrics.map_metric,
        device=config.DEVICE)
    print(f'SVD FASDD\n{val_metrics}')
    val_metrics = eval_fn(
        loader=detection_loader, 
        model=pruned_after_svd_model,                         
        metric=metrics.map_metric,
        device=config.DEVICE)
    print(f'Pruned after SVD FASDD\n{val_metrics}')

Validating: 100%|████████████████████████████████████████████████████████████████████████████████| 361/361 [00:29<00:00, 12.30it/s]


Val mAP = 0.6405
BED FASDD
{'mAP': tensor(0.6405), 'AP': [0.6541241407394409, 0.6267908811569214], 'AR': [0.7165079712867737, 0.6754377484321594]}


Validating: 100%|████████████████████████████████████████████████████████████████████████████████| 361/361 [00:28<00:00, 12.48it/s]


Val mAP = 0.6186
SVD FASDD
{'mAP': tensor(0.6186), 'AP': [0.6275915503501892, 0.6096052527427673], 'AR': [0.6997619867324829, 0.6654180884361267]}


Validating: 100%|████████████████████████████████████████████████████████████████████████████████| 361/361 [00:28<00:00, 12.68it/s]


Val mAP = 0.6289
Pruned after SVD FASDD
{'mAP': tensor(0.6289), 'AP': [0.6367916464805603, 0.6209565997123718], 'AR': [0.7040122151374817, 0.6724412441253662]}


# Export to ONNX

In [16]:
bed_model.to('cpu')
torch_input = torch.rand(1, 3, 224, 224)
torch.onnx.export(bed_model, torch_input, './onnx_models/bed_detection_fasdd__cpu.onnx') 

In [17]:
svd_model.to('cpu')
torch_input = torch.rand(1, 3, 224, 224)
torch.onnx.export(svd_model, torch_input, './onnx_models/bed_detection_fasdd__svd__cpu.onnx') 

In [18]:
pruned_after_svd_model.to('cpu')
torch_input = torch.rand(1, 3, 224, 224)
torch.onnx.export(pruned_after_svd_model, torch_input, './onnx_models/bed_detection_fasdd__pruned_after_svd__cpu.onnx') 